# Phoenics: a universal deep Bayesian optimizer

This notebook introduces Phoenics and demonstrates how to use Phoenics for simple optimization problems on parameter domains of arbitrary dimensionality. For further details and more examples, please check out the GitHub repository:

[https://github.com/aspuru-guzik-group/phoenics](https://github.com/aspuru-guzik-group/phoenics)

Details on the mathematical methods and models implemented in Phoenics can be found at:

F. Hase, L.M. Roch, C. Kreisbeck, A. Aspuru-Guzik, Phoenics: a universal deep Bayesian optimizer. (2018) [https://arxiv.org/abs/1801.01469](https://arxiv.org/abs/1801.01469)

In [1]:
# import python libraries
import sys, json, copy
sys.path.append('../')

import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_context('paper', font_scale = 2.0, rc = {'lines.linewidth': 3})
sns.set_style('ticks')
%matplotlib inline

# import phoenics
from phoenics.phoenics import Phoenics

## The configuration file

Parameters for an optimization procedure need to be provided in a **configuration file** in the jSON format. The configuration file needs to contain general information about the optimization procedure as well as specific information about the variables for which the optimum is to be found.

Below is an example of a configuration file for an optimization run with two variables, `param0` and `param1`.

In [2]:
# check the configuration file
config_file = 'my_experiment.txt'
print(open(config_file, 'r').read())

{
	"general": {
		"num_batches": 1,
		"batch_size": 3
	},
	"variables": [{"param0": { "low": -4.0,  "high": 2.0,   "type": "float",  "size": 2} }, 
				  {"param1": { "low": 30.0,  "high": 60.0,  "type": "float",  "size": 3} }]
}



The `'general'` keyword is used to define general information about the optimization procedure:
* `num_batches`: The default number of parameter sets to be proposed in one optimization iteration
* `batch_size`: The default number of parameter sets proposed from different sampling strategies in a single batch

The '`variables'` keyword is used to define specific information about all variables which need to be optimized. This keyword expects a list of all variables on which the optimization procedure is supposed to be run:
* '`param0`': example for the unique name of a variable. You may choose your favorite names for all variables, but variable names **must** be unique!
* '`low`': lower bound for this variable (inclusive)
* '`high`': upper bound for this variable (inclusive)
* '`type`': variable type (only `'float'` is supported at this point)
* '`size`': size of this variable; must be an integer greater than or equal to 1

## Initializing an instance of Phoenics

The configuration file now allows for easy initialization of an instance of Phoenics as it is demonstrated in the cell below

In [3]:
# create an instance of Phoenics
phoenics = Phoenics(config_file)

Phoenics automatically parses all information provided in the configuration file. 

In [4]:
# phoenics automatically parses the configuration file
print(phoenics.param_dict)

{'general': {'num_batches': 1, 'batch_size': 3}, 'variables': [{'param0': {'low': -4.0, 'high': 2.0, 'type': 'float', 'size': 2}}, {'param1': {'low': 30.0, 'high': 60.0, 'type': 'float', 'size': 3}}]}


## Sampling new parameter sets for querying the objective

An instance of Phoenics can then be used to propose new parameter sets at which the objective function should be evaluated to discover the global optimimum. Phoenics can run in two different modes, corresponding to two different scenarios:

* **no observations available**: This case might correspond to scenarios in which the experiment is just initialized. The objective function has not been evaluated yet and no prior knowledge about the landscape is available. In this case, Phoenics acts as a uniform random sampler for all declared parameters

* **observations available**: In this case, the objective function has been evaluated at a number of parameter points already, which provides information about the landscape of the objective function. Phoenics exploits this knowledge to propose new parameter points for querying the objective function and discovering the global optimum

### No observations available

First, let's consider a case in which we do not have any knowledge about the objective function landscape. Parameter sets can always be requested with the `choose` method. Simply calling the `choose` method without providing any additional arguments generates as many parameter sets as were specified with the `num_batches` option in the configuration file

In [5]:
# phoenics is a uniform random sampler without any observations
sampled_params = phoenics.choose()
print(sampled_params)

[{'param0': {'samples': array([-1.89323638, -1.51087277])}, 'param1': {'samples': array([ 34.01491254,  52.79273062,  44.30966735])}}]


The `choose` method returns a list of python dictionaries with information about the sampled parameter points.

Phoenics also allows us to deviate from the initial settings. With the `num_samples` argument, the number of proposed parameter sets can be changed.

In [6]:
np.random.seed(100691)
# you can get as many samples as you like
sampled_params = phoenics.choose(num_samples = 3)
for sample in sampled_params:
    print(sample)

{'param0': {'samples': array([ 1.95654646, -1.10101855])}, 'param1': {'samples': array([ 42.95136195,  40.99273375,  44.48832017])}}
{'param0': {'samples': array([-1.72039787, -2.17098636])}, 'param1': {'samples': array([ 55.71303832,  49.65954287,  34.3424954 ])}}
{'param0': {'samples': array([ 1.06541465,  0.94323606])}, 'param1': {'samples': array([ 36.57336715,  53.37726152,  56.63508027])}}


The format of returning sampled parameter points allows for easy matching of sampled numbers and corresponding variables. However, should this option be inconvenient for you, you can also ask Phoenics to return the sampled parameters as a numpy array. To do so, simply call `choose` with the `as_array` argument set to `True`. Note, that elements along the second dimension are listed in their order of appearance in the configuration file

In [7]:
np.random.seed(100691)
# phoenics can also return just parameter arrays
sampled_params_array = phoenics.choose(num_samples = 3, as_array = True)
for sample in sampled_params_array:
    print(sample)

[  1.95654646  -1.10101855  42.95136195  40.99273375  44.48832017]
[ -1.72039787  -2.17098636  55.71303832  49.65954287  34.3424954 ]
[  1.06541465   0.94323606  36.57336715  53.37726152  56.63508027]


### Observations available

Running Phoenics when information about the objective function are available. Phoenics expects observations to be provided in a similar format as sampled parameter sets are returned by the `choose` method.

Observations need to be provided as a list of dictionaries, where for each dictionary the key is the unique variable name, the observed parameter values for this variable are stored at `'samples'` and the observed objective function value is stored at `'loss'`. An example for how to generate a valid list of observations is provided below

In [8]:
# now let's fake a loss function for phoenics
evaluated_params = copy.deepcopy(sampled_params)
prior_losses = []
for sample_dict in evaluated_params:
    # calculate a dummy loss 
    loss = np.linalg.norm(sample_dict['param0']['samples']) + np.linalg.norm(sample_dict['param1']['samples'])
    prior_losses.append(loss)
    # store loss in the dictionary at the 'loss' keyword
    sample_dict['loss'] = loss

Let's check the generated list of observations

In [9]:
for evaluation in evaluated_params:
    print(evaluation)

{'param0': {'samples': array([ 1.95654646, -1.10101855])}, 'param1': {'samples': array([ 42.95136195,  40.99273375,  44.48832017])}, 'loss': 76.436939989369804}
{'param0': {'samples': array([-1.72039787, -2.17098636])}, 'param1': {'samples': array([ 55.71303832,  49.65954287,  34.3424954 ])}, 'loss': 84.924861862048374}
{'param0': {'samples': array([ 1.06541465,  0.94323606])}, 'param1': {'samples': array([ 36.57336715,  53.37726152,  56.63508027])}, 'loss': 87.412929138315519}


**Important**: Phoenics currently only supports **minimizations**! If you would rather like to maximize a certain objective, make sure to provide the negative objective function value.

The list of observations can now be used to request new parameter points for querying the objective function taking into account the acquired knowledge about the objective function landscape.

In [ ]:
# phoenics now starts sampling
new_params = phoenics.choose(observations = evaluated_params, as_array = True)

# running density estimation


In [ ]:
# let's compute the losses again
post_losses = []
for sample in new_params[0]:
    # calculate a dummy loss 
    loss = np.linalg.norm(sample[0:2]) + np.linalg.norm(sample[3:])
    post_losses.append(loss)
    # store loss in the dictionary at the 'loss' keyword

In [ ]:
# and plot the results
plt.plot(np.arange(3), prior_losses, marker = 'o', ls = '', color = 'k', markersize = 12, label = 'prior')
plt.plot(np.arange(3) + 3, post_losses, marker = 'o', ls = '', markersize = 12, label = 'post')
plt.ylim(0, 92)
plt.legend()
plt.xlabel('# evaluations')
plt.ylabel('loss')

## Summary

A typical optimization workflow with Phoenics could consist of the following steps:
* generate your configuration file to provide general information and information about the variables to optimize
* initialize the optimization procedure by running Phoenics without any observations (optional)
* provide a list of observations to Phoenics to exploit acquired knowledge

For questions and/or support, please don't hesitate to contact Flo (fhase@g.harvard.edu)